<a href="https://colab.research.google.com/github/FahadHassan100/agentic-ai-practice/blob/main/RAG_Text_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
list(genai.list_models())

**Making example How to embed the text.**

In [ ]:
from typing import Dict

result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content=[
        "What is the meaning of life?",
        "How much wood would at woodchuck chuck",
        "How dose the brain work?",
    ],
    task_type="retrieval_document",
    title="Embedding of the list of strings",
)

# A list of inputs > A list of vectors output
for v in result["embedding"]:
  print(str(v)[:5], "... TRIMMED ...", len(v))

# Building Vector Store & Retreival using Chroma DB

In [ ]:
!pip install -Uq langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.3 MB/s eta 0:00:00


In [ ]:
import getpass
import os

In [ ]:
from langchain_core.documents import Document

documents=[
        Document(page_content="Dogs are great companions, know for there loyality and friendliness", metadata={"source": "mammal-pets-doc"}),
        Document(page_content="Cats are independent pets that often enjoy there own space.", metadata={"source": "mammal-pets-doc"}),
        Document(page_content="Rabbits are social animal that need plently of space to hop around.", metadata={"source": "mammal-pets-doc"}),
]

In [ ]:
!pip install -Uq langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=userdata.get("GOOGLE_API_KEY"))

In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings,
)

In [ ]:
list(dir(vectorstore))

In [ ]:
vectorstore

In [ ]:
vectorstore.similarity_search("Cat")

In [ ]:
await vectorstore.asimilarity_search("Cat")

In [ ]:
vectorstore.similarity_search_with_score("Cat")

**Just search in Vector DB from 1 word**

In [ ]:
embedding = embeddings.embed_query("Cat") #Convert cat into vactor
vectorstore.similarity_search_by_vector(embedding)

**Retrievers**

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) #Select top Result

retriever.batch(['shark'])

In [ ]:
!pip install -Uq langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=userdata.get("GOOGLE_API_KEY")
    )

**Until here The first part of RAG which is Retrieval is completed. Now we are moving to second part Augmented  **

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Anwser this question using the provided context only,

{question}

Context:
{context}

"""

In [ ]:
prompt = ChatPromptTemplate.from_messages([('Human', message)])

In [ ]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [ ]:
response = rag_chain.invoke("What is a cat?")

print(response.content)

# Now use google gemini embedding model for retriver


**Face detection with embedding**

In [ ]:
!pip install -Uq facenet-pytorch pillow

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1

In [ ]:
model = InceptionResnetV1(pretrained='vggface2').eval()
model

In [ ]:
# Preprocessing function to transform the image into a tensor
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

def create_image_embedding(image_path):
  try:
    input_tensor = preprocess_image(image_path)
    with torch.no_grad():
      embeddings = model(input_tensor)
    return embeddings.squeeze().numpy()
  except Exception as e:
    print("Error:", e)
    return None

In [ ]:
!mkdir images

In [ ]:
import requests
import os

def save_image_from_url(image_url, image_name):
  """
  Downloads an image from a URL and save it to the 'images' folder.

  Args:
    image_url: The URL of the image to download.
    image_name: The name of the file to save the image as.
  """
  try:
    if not os.path.exists('images'):
      os.makedirs('images')

    image_path = os.path.join('images', image_name)
    response = requests.get(image_url, stream=True)
    response.raise_for_status()

    with open(image_path, 'wb') as image_file:
      for chunk in response.iter_content(chunk_size=8192):
        image_file.write(chunk)

    print(f"Image saved to {image_path}")
  except requests.exceptions.RequestException as e:
    print(f"Error downloading image: {e}")
  except Exception as e:
    print(f"Error saving image: {e}")

In [ ]:
save_image_from_url("<image_url>")
save_image_from_url("<image_url>")
save_image_from_url("<image_url>")

In [ ]:
# Example usage
image_path = "./images/image_name.jpg"
q2 = create_image_embedding(image_path)

print("Image Embedding Shape:", q2.shape)
print("Image Embedding:", q2)

In [ ]:
image_path = "./images/s2.jpg"
q2 = create_image_embedding(image_path)

print("Image Embedding Shape:", q2.shape)
print("Image Embedding:", q2)

In [ ]:
q1 = create_image_embedding("./images/q1.jpg")
q2 = create_image_embedding("./images/q2.jpg")
s1 = create_image_embedding("./images/s1.jpg")
s2 = create_image_embedding("./images/s2.jpg")
z1 = create_image_embedding("./images/z1.jpg")
z2 = create_image_embedding("./images/z2.jpg")

In [ ]:
!pip install -U milvus-lite

!pip install -U pymilvus

In [ ]:
from pymilvus import MilvusClient
client = MilvusClient('./milvus_demo.db')

In [ ]:
from pymilvus import MilvusClient
import numpy as np


client = MilvusClient('./milvus_demo.db')
client.create_collection(
    collection_name="images",
    dimension=512,
)

In [ ]:
data = [
    {"id": 1, "person_name": "Qasim", "vector": q1},
    {"id": 2, "person_name": "Qasim", "vector": q2},
    {"id": 3, "person_name": "Shahzad", "vector": s1},
    {"id": 4, "person_name": "Shahzad", "vector": s2},
    {"id": 5, "person_name": "Zia Khan", "vector": z1},
    {"id": 6, "person_name": "Zia Khan", "vector": z2},
]

**Now Storing into Database**

In [ ]:
res = client.insert(
    collection_name="images",
    data=data,
)

In [ ]:
res = client.search(
    collection_name="images",
    data=[s1],
    limit=1,
    output_fields=["id","person_name"]
)
print(res)

In [ ]:
q3 = create_image_embedding("./images/q3.jpg")
q3

In [ ]:
res = client.search(
    collection_name="images",
    data=[q3],
    limit=1,
    output_fields=["id","person_name"]
)
print(res)